In [1]:
import warnings
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from iterative_iv_plr import normal_expectile, ExpectileRegression, NonlinearExpectileRegression
from iterative_iv_plr import fit_plr_ivtype_iterative_weighted

face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')

warnings.filterwarnings("ignore")

In [4]:
#partially linear

from scipy.linalg import toeplitz

_array_alias = ['array', 'np.ndarray', 'np.array', np.ndarray]
_data_frame_alias = ['DataFrame', 'pd.DataFrame', pd.DataFrame]
_dml_data_alias = ['DoubleMLData']

def make_plr_expectile(n_obs=500, dim_x=20, alpha=0.5, return_type="array", **kwargs):

    cov_mat = toeplitz([np.power(0.7, k) for k in range(dim_x)])
    
    x = np.random.standard_normal(size=(n_obs, dim_x))

    d = (
        100 * x[:, 0]**2
        + 1000 * (np.exp(x[:, 2]) / (1 + np.exp(x[:, 2])))
        + np.random.standard_normal(size=n_obs)
    )
    
    zeta = np.random.normal(loc=0.0, scale = 0.7 * d)
    g_true = 100 * (np.exp(x[:, 0]) / (1 + np.exp(x[:, 0]))) + 1000 * x[:, 2]
    y = (
        alpha * d
        + g_true
        + zeta
    )

    if return_type in _array_alias:
        return x, y, d, g_true

    elif return_type in _data_frame_alias + _dml_data_alias:
        x_cols = [f"X{i + 1}" for i in range(dim_x)]
        data = pd.DataFrame(np.column_stack((x, y, d)), columns=x_cols + ["y", "d"])
        if return_type in _data_frame_alias:
            return data
        else:
            return {"data": data, "y_col": "y", "d_col": "d", "x_cols": x_cols}

    else:
        raise ValueError("Invalid return_type.")

np.random.seed(123)
n_rep = 1000
n_obs = 1000
n_vars = 20
alpha = 0.5

data = list()

for i_rep in range(n_rep):
    (x, y, d,g_true) = make_plr_expectile(alpha=alpha, n_obs=n_obs, dim_x=n_vars, return_type='array')
    data.append((x, y, d,g_true))


In [9]:
from iterative_iv_plr import fit_plr_nonorthogonal_weighted
from lightgbm import LGBMRegressor
np.random.seed(3333)
save_dir = "LGBM_biased"
os.makedirs(save_dir, exist_ok=True)

taus = [0.1, 0.5, 0.9]
simulation_results = {}

for tau in taus:
    print(f"Running tau = {tau:.1f}")
    mu=normal_expectile(tau)
    ml = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=3,
        num_leaves=8,
        min_child_samples=50,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=2025,
        verbosity=-1
    )

    theta_dml = np.full(n_rep, np.nan)
    se_dml = np.full(n_rep, np.nan)

    for i_rep in range(n_rep):
        print(f'\tReplication {i_rep+1}/{n_rep}', end='\r')
        x, y, d, g_true = data[i_rep]
        theta_hat, se_hat, extras = fit_plr_nonorthogonal_weighted(
            x=x,
            y=y,
            d=d,
            ml=ml,
            tau=tau,
            max_iter=10,
            tol=1e-7,
            n_folds=2,
            theta0=0.0
        )

        theta_dml[i_rep] = theta_hat
        se_dml[i_rep] = se_hat

    simulation_results[f"tau_{tau:.1f}"] = {
        "theta_dml": theta_dml,
        "se_dml": se_dml,
        "mu": mu
    }

np.savez(
    os.path.join(save_dir, "simulation_results.npz"),
    **simulation_results
)

Running tau = 0.1
Running tau = 0.5/1000
Running tau = 0.9/1000
	Replication 1000/1000

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import FixedLocator

plt.rcParams.update({
    "text.usetex": True,  
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"""
        \usepackage{times}
        \usepackage{bm}
        \usepackage{mathptmx}
    """,
    "axes.unicode_minus": False
})

def bold_formatter(x, pos):
    if x == int(x):
        return r'$\mathbf{' + f'{int(x)}' + r'}$'
    else:
        return r'$\mathbf{' + f'{x:.1f}' + r'}$'

save_dir = "LGBM_biased"
os.makedirs(save_dir, exist_ok=True)
loaded = np.load(
    os.path.join(save_dir, "simulation_results.npz"),
    allow_pickle=True
)
taus = [0.1, 0.5, 0.9]

for tau in taus:
    res = loaded[f"tau_{tau:.1f}"].item()
    theta_dml = res["theta_dml"]
    se_dml = res["se_dml"]
    mu = res["mu"]
    
    fig_dml, ax = plt.subplots(constrained_layout=True)
    sns.histplot(
        (theta_dml - alpha - 0.7 * mu) / se_dml,
        stat='density',
        bins=30,
        label=r'\textbf{Simulation}',
        ax=ax
    )
    ax.yaxis.set_major_locator(
        FixedLocator(np.arange(0, 1.3, 0.1))
    )
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.axvline(0.0, color='k', linewidth=1.5)
    xx = np.arange(-5, 5, 0.001)
    yy = stats.norm.pdf(xx)
    ax.plot(xx, yy, color='k', linewidth=1.5, label=r'$\bm{\mathcal{N}(0,1)}$')
    ax.set_title(
        r'\textbf{Non-Orthogonal}',
        fontsize=18
    )
    ax.set_xlabel(
        r'$\bm{(\hat{\theta}^{\tau}_{0} - \theta^{\tau}_{0})/\hat{\sigma}}$',
        fontsize=16
    )
    ax.set_ylabel(
        r'\textbf{Density}',
        fontsize=16
    )

    ax.xaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.tick_params(axis='both', which='major', labelsize=14)
    leg = ax.legend(frameon=False, fontsize=14)
    ax.set_xlim([-20, 20])
    filename = f"{save_dir}/bLGBMtau{int(tau*10)}.eps"
    fig_dml.savefig(
        filename,
        format='eps',
        dpi=300,
        bbox_inches='tight'
    )
    plt.close(fig_dml)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.


In [ ]:
from lightgbm import LGBMRegressor
np.random.seed(3333)
save_dir = "LGBM_debiased"
os.makedirs(save_dir, exist_ok=True)

taus = [0.1, 0.3, 0.5, 0.7, 0.9]

simulation_results = {}
for tau in taus:
    print(f"Running tau = {tau:.1f}")
    mu = normal_expectile(tau)
    ml_m = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=3,
        num_leaves=8,
        min_child_samples=50,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=2025,
        verbosity=-1
    )

    ml_g = NonlinearExpectileRegression(tau=tau)

    theta_dml = np.full(n_rep, np.nan)
    se_dml = np.full(n_rep, np.nan)

    for i_rep in range(n_rep):
        print(f'\tReplication {i_rep+1}/{n_rep}', end='\r')
        x, y, d, g_true = data[i_rep]

        theta_hat, se_hat, extras = fit_plr_ivtype_iterative_weighted(
            x=x, y=y, d=d,
            ml_m=ml_m,
            ml_g=ml_g,
            n_folds=2,
            tau=tau,
            max_iter=10,
            tol=1e-7,
            theta0=0.0,
            random_state=2025,
            n_jobs=-1
        )

        theta_dml[i_rep] = theta_hat
        se_dml[i_rep] = se_hat
  

    simulation_results[f"tau_{tau:.1f}"] = {
        "theta_dml": theta_dml,
        "se_dml": se_dml,
        "mu": mu  
    }

np.savez(os.path.join(save_dir, "simulation_results.npz"), **simulation_results)

In [ ]:
from lightgbm import LGBMRegressor
np.random.seed(3333)
save_dir = "LGBM_debiased"
os.makedirs(save_dir, exist_ok=True)

taus = [0.1, 0.3, 0.5, 0.7, 0.9]

simulation_results = {}
for tau in taus:
    print(f"Running tau = {tau:.1f}")
    mu = normal_expectile(tau)
    ml_m = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=3,
        num_leaves=8,
        min_child_samples=50,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=2025,
        verbosity=-1
    )

    ml_g = NonlinearExpectileRegression(tau=tau)

    theta_dml = np.full(n_rep, np.nan)
    se_dml = np.full(n_rep, np.nan)

    for i_rep in range(n_rep):
        print(f'\tReplication {i_rep+1}/{n_rep}', end='\r')
        x, y, d, g_true = data[i_rep]

        theta_hat, se_hat, extras = fit_plr_ivtype_iterative_weighted(
            x=x, y=y, d=d,
            ml_m=ml_m,
            ml_g=ml_g,
            n_folds=2,
            tau=tau,
            max_iter=10,
            tol=1e-7,
            theta0=0.0,
            random_state=2025,
            n_jobs=-1
        )

        theta_dml[i_rep] = theta_hat
        se_dml[i_rep] = se_hat
  

    simulation_results[f"tau_{tau:.1f}"] = {
        "theta_dml": theta_dml,
        "se_dml": se_dml,
        "mu": mu  
    }

np.savez(os.path.join(save_dir, "simulation_results.npz"), **simulation_results)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import os
from matplotlib.ticker import FuncFormatter

save_dir = "LGBM_debiased"
os.makedirs(save_dir, exist_ok=True)

plt.rcParams.update({
    "text.usetex": True,  
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"""
        \usepackage{times}
        \usepackage{bm}
        \usepackage{mathptmx}
    """,
    "axes.unicode_minus": False
})

def bold_formatter(x, pos):
    if abs(x - round(x)) < 1e-10:
        return r'$\mathbf{' + f'{int(round(x))}' + r'}$'
    else:
        formatted = f'{x:.2f}'.rstrip('0').rstrip('.')
        return r'$\mathbf{' + formatted + r'}$'

loaded = np.load(
    os.path.join(save_dir, "simulation_results.npz"),
    allow_pickle=True
)

taus = [0.1, 0.3, 0.5, 0.7, 0.9]

for tau in taus:
    res = loaded[f"tau_{tau:.1f}"].item()
    theta_dml = res["theta_dml"]
    se_dml = res["se_dml"]
    mu = res["mu"]
    fig_dml, ax = plt.subplots(constrained_layout=True)
    
    sns.histplot(
        (theta_dml - alpha - 0.7 * mu) / se_dml,
        stat='density',
        binwidth=0.3,
        binrange=(-6, 6),   
        edgecolor='black',  
        linewidth=1.0,
        label=r'\textbf{Simulation}',
        ax=ax
    )
    
    ax.axvline(0.0, color='k', linewidth=1.5)
    xx = np.arange(-5, 5, 0.001)
    yy = stats.norm.pdf(xx)
    ax.plot(xx, yy, color='k', linewidth=1.5, label=r'$\bm{\mathcal{N}(0,1)}$')
    
    ax.set_title(
        r'\textbf{Orthogonal}',
        fontsize=18
    )

    ax.set_xlabel(
        r'$\bm{(\hat{\theta}^{\tau}_{0} - \theta^{\tau}_{0})/\hat{\sigma}}$',
        fontsize=16
    )
    ax.set_ylabel(
        r'\textbf{Density}',
        fontsize=16
    )

    ax.set_xlim([-6.0, 6.0])
    ax.xaxis.set_major_locator(plt.MaxNLocator(nbins=7, integer=False))
    ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])
    ax.xaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    leg = ax.legend(frameon=False, fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=14)
    filename = f"{save_dir}/dLGBMtau{int(tau*10)}.eps"
    fig_dml.savefig(
        filename,
        format='eps',
        dpi=300,
        bbox_inches='tight'
    )
    plt.close(fig_dml)

In [2]:
#high-dimensional linear

import numpy as np
from scipy.linalg import toeplitz
import pandas as pd

def make_plr_linear_positive_highdim(
    n_obs=500,
    dim_x=100,
    alpha=0.5,
    noise_sd=1.0,
    seed=None,
    batch_factor=5
):

    if seed is not None:
        np.random.seed(seed)
        
    cov_mat = toeplitz([0.7 ** k for k in range(dim_x)])

    X_list, d_list, g_list, y_list = [], [], [], []

    collected = 0
    batch_size = int(batch_factor * n_obs)

    while collected < n_obs:
        X_batch = np.random.multivariate_normal(
            mean=np.zeros(dim_x),
            cov=cov_mat,
            size=batch_size
        )
        eps = np.random.normal(0.0, noise_sd, size=batch_size)
        d_batch = X_batch[:, :10].sum(axis=1) + eps
        mask = d_batch > 0
        if not np.any(mask):
            continue

        X_acc = X_batch[mask]
        d_acc = d_batch[mask]
        g_acc = X_acc[:, 10:20].sum(axis=1)
        zeta = np.random.normal(0.0, 0.7 * d_acc)
        y_acc = alpha * d_acc + g_acc + zeta

        X_list.append(X_acc)
        d_list.append(d_acc)
        g_list.append(g_acc)
        y_list.append(y_acc)

        collected += X_acc.shape[0]

    X = np.vstack(X_list)[:n_obs]
    d = np.concatenate(d_list)[:n_obs]
    g_true = np.concatenate(g_list)[:n_obs]
    y = np.concatenate(y_list)[:n_obs]

    return X, y, d, g_true

np.random.seed(123)
n_rep = 1000
n_obs = 1000
n_vars = 200
alpha = 0.5

data = []

for i_rep in range(n_rep):
    X, y, d, g_true = make_plr_linear_positive_highdim(
        n_obs=n_obs,
        dim_x=n_vars,
        alpha=alpha,
        seed=i_rep
    )
    data.append((X, y, d, g_true))



In [4]:
from sklearn.linear_model import Lasso
from iterative_iv_plr import fit_plr_nonorthogonal_weighted

np.random.seed(3333)
save_dir = "Lasso_biased"
os.makedirs(save_dir, exist_ok=True)

taus = [0.1, 0.5, 0.9]
simulation_results = {}

for tau in taus:
    print(f"Running tau = {tau:.1f}")
    mu = normal_expectile(tau)
    ml =  LassoCV(
        alphas = [ 0.0001, 0.001,  0.01,  0.1,  1.0, 10.0],
        cv=3,                 
        max_iter=1000,
        tol=1e-4,
        n_jobs=-1,
        selection='random',
        random_state=2025,
        verbose=False         
    )

    theta_dml = np.full(n_rep, np.nan)
    se_dml = np.full(n_rep, np.nan)

    for i_rep in range(n_rep):
        print(f'\tReplication {i_rep+1}/{n_rep}', end='\r')
        x, y, d, g_true = data[i_rep]
        theta_hat, se_hat, extras = fit_plr_nonorthogonal_weighted(
            x=x,
            y=y,
            d=d,
            ml=ml,
            tau=tau,
            max_iter=10,
            tol=1e-7,
            n_folds=2,
            theta0=0.0
        )

        theta_dml[i_rep] = theta_hat
        se_dml[i_rep] = se_hat

    simulation_results[f"tau_{tau:.1f}"] = {
        "theta_dml": theta_dml,
        "se_dml": se_dml,
        "mu": mu
    }

np.savez(
    os.path.join(save_dir, "simulation_results.npz"),
    **simulation_results
)


Running tau = 0.1
Running tau = 0.5/1000
Running tau = 0.9/1000
	Replication 1000/1000

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import FixedLocator

plt.rcParams.update({
    "text.usetex": True,  
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"""
        \usepackage{times}
        \usepackage{bm}
        \usepackage{mathptmx}
    """,
    "axes.unicode_minus": False
})

def bold_formatter(x, pos):
    if x == int(x):
        return r'$\mathbf{' + f'{int(x)}' + r'}$'
    else:
        return r'$\mathbf{' + f'{x:.1f}' + r'}$'

save_dir = "Lasso_biased"
os.makedirs(save_dir, exist_ok=True)
loaded = np.load(
    os.path.join(save_dir, "simulation_results.npz"),
    allow_pickle=True
)
taus = [0.1, 0.5, 0.9]
for tau in taus:
    res = loaded[f"tau_{tau:.1f}"].item()
    theta_dml = res["theta_dml"]
    se_dml = res["se_dml"]
    mu = res["mu"]
    
    fig_dml, ax = plt.subplots(constrained_layout=True)
    sns.histplot(
        (theta_dml - alpha - 0.7 * mu) / se_dml,
        stat='density',
        bins=30,
        label=r'\textbf{Simulation}',
        ax=ax
    )
    ax.yaxis.set_major_locator(
        FixedLocator(np.arange(0, 1, 0.1))
    )
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.axvline(0.0, color='k', linewidth=1.5)
    xx = np.arange(-5, 5, 0.001)
    yy = stats.norm.pdf(xx)
    ax.plot(xx, yy, color='k', linewidth=1.5, label=r'$\bm{\mathcal{N}(0,1)}$')
    ax.set_title(
        r'\textbf{Non-Orthogonal}',
        fontsize=18
    )
    ax.set_xlabel(
        r'$\bm{(\hat{\theta}^{\tau}_{0} - \theta^{\tau}_{0})/\hat{\sigma}}$',
        fontsize=16
    )
    ax.set_ylabel(
        r'\textbf{Density}',
        fontsize=16
    )
    ax.xaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.tick_params(axis='both', which='major', labelsize=14)
    leg = ax.legend(frameon=False, fontsize=14)
    ax.set_xlim([-20, 20])
    filename = f"{save_dir}/bLassotau{int(tau*10)}.eps"
    fig_dml.savefig(
        filename,
        format='eps',
        dpi=300,
        bbox_inches='tight'
    )
    plt.close(fig_dml)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Failed to find a Ghostscript installation.  Distillation step skipped.


In [ ]:
from sklearn.linear_model import LassoCV
import os
np.random.seed(3334)
save_dir = "Lasso_debiased"
os.makedirs(save_dir, exist_ok=True)

taus = [0.1, 0.3, 0.5, 0.7, 0.9]
simulation_results = {}

for tau in taus:
    print(f"Running tau = {tau:.1f}")
    mu=normal_expectile(tau)
    ml_m =  LassoCV(
        alphas = [ 0.0001, 0.001,  0.01,  0.1,  1.0, 10.0],
        cv=3,                 
        max_iter=1000,
        tol=1e-4,
        n_jobs=-1,
        selection='random',
        random_state=2025,
        verbose=False        
    )
    ml_g = ExpectileRegression(tau=tau, reg_lambda=0.01)

    theta_dml = np.full(n_rep, np.nan)
    se_dml = np.full(n_rep, np.nan)

    for i_rep in range(n_rep):
        print(f'\tReplication {i_rep+1}/{n_rep}', end='\r')
        x, y, d, g_true = data[i_rep]
        theta_hat, se_hat, extras = fit_plr_ivtype_iterative_weighted(
            x=x, y=y, d=d,
            ml_m=ml_m,
            ml_g=ml_g,
            n_folds=2,
            tau=tau,
            max_iter=10,
            tol=1e-7,
            theta0=0.0,
            random_state=2025,
            n_jobs=-1
        )

        theta_dml[i_rep] = theta_hat
        se_dml[i_rep] = se_hat

    simulation_results[f"tau_{tau:.1f}"] = {
        "theta_dml": theta_dml,
        "se_dml": se_dml,
        "mu": mu  
    }

np.savez(os.path.join(save_dir, "simulation_results.npz"), **simulation_results)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import os
from matplotlib.ticker import FuncFormatter

save_dir = "Lasso_debiased"
os.makedirs(save_dir, exist_ok=True)
plt.rcParams.update({
    "text.usetex": True,  
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"""
        \usepackage{times}
        \usepackage{bm}
        \usepackage{mathptmx}
    """,
    "axes.unicode_minus": False
})

def bold_formatter(x, pos):
    if abs(x - round(x)) < 1e-10:
        return r'$\mathbf{' + f'{int(round(x))}' + r'}$'
    else:
        formatted = f'{x:.2f}'.rstrip('0').rstrip('.')
        return r'$\mathbf{' + formatted + r'}$'
loaded = np.load(
    os.path.join(save_dir, "simulation_results.npz"),
    allow_pickle=True
)
taus = [0.1, 0.3, 0.5, 0.7, 0.9]
for tau in taus:
    res = loaded[f"tau_{tau:.1f}"].item()
    theta_dml = res["theta_dml"]
    se_dml = res["se_dml"]
    mu = res["mu"]
    fig_dml, ax = plt.subplots(constrained_layout=True)
    sns.histplot(
        (theta_dml - alpha - 0.7 * mu) / se_dml,
        stat='density',
        binwidth=0.3,
        binrange=(-6, 6),   
        edgecolor='black',  
        linewidth=1.0,
        label=r'\textbf{Simulation}',
        ax=ax
    )
    
    ax.axvline(0.0, color='k', linewidth=1.5)
    xx = np.arange(-5, 5, 0.001)
    yy = stats.norm.pdf(xx)
    ax.plot(xx, yy, color='k', linewidth=1.5, label=r'$\bm{\mathcal{N}(0,1)}$')
    ax.set_title(
        r'\textbf{Orthogonal}',
        fontsize=18
    )
    
    ax.set_xlabel(
        r'$\bm{(\hat{\theta}^{\tau}_{0} - \theta^{\tau}_{0})/\hat{\sigma}}$',
        fontsize=16
    )
    ax.set_ylabel(
        r'\textbf{Density}',
        fontsize=16
    )
    
    ax.set_xlim([-6.0, 6.0])
    ax.xaxis.set_major_locator(plt.MaxNLocator(nbins=7, integer=False))
    ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])
    ax.xaxis.set_major_formatter(FuncFormatter(bold_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(bold_formatter))
    leg = ax.legend(frameon=False, fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=14)
    filename = f"{save_dir}/dLassotau{int(tau*10)}.eps"
    fig_dml.savefig(
        filename,
        format='eps',
        dpi=300,
        bbox_inches='tight'
    )
    plt.close(fig_dml)